# Auswertung der Parquet Dateien aus dem Echtzeitarchiv V14

## Import der Module und Setzen Parameter

In [53]:
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import openpyxl

import datetime as dt

import shutil

from dotenv import dotenv_values
from redmine import delete_upload_dmsf
import logging
import glob

from openpyxl.styles import NamedStyle

In [ ]:
log_file = "log/log_rt.txt"
logging.basicConfig(filename=log_file, 
                        level=logging.INFO,
                        style="{",
                        format="{asctime} [{levelname:8}] {message}",
                        datefmt="%d.%m.%Y %H:%M:%S")

load_dotenv()

In [4]:
sys.path.append('/home/zvbn/python/rt2')

In [5]:
from class_rt_duck import RtDuck

In [6]:
logging.info("Auswertung RT aus parquet gestartet")

In [7]:
config = dotenv_values(".env")
#config

In [8]:
pd.options.display.max_columns = 100

In [ ]:
jetzt = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
gestern= (dt.date.today() - dt.timedelta(1)).strftime('%Y-%m-%d')
letzte07tage= (dt.date.today() - dt.timedelta(7)).strftime('%Y-%m-%d')
letzte14tage= (dt.date.today() - dt.timedelta(14)).strftime('%Y-%m-%d')
letzte21tage= (dt.date.today() - dt.timedelta(21)).strftime('%Y-%m-%d')

print(jetzt, letzte21tage)

## Funktionen

In [10]:
def replace_german_special_characters(text) -> str:
    replacements = {
        'ä': 'ae',
        'ö': 'oe',
        'ü': 'ue',
        'Ä': 'Ae',
        'Ö': 'Oe',
        'Ü': 'Ue',
        'ß': 'ss'
    }
    
    for german_char, replacement in replacements.items():
        text = text.replace(german_char, replacement)
    
    return text

In [11]:
#für die Formatierung der Ausgabe in html
func_proz = lambda s: str(int((1-s) * 1000)/10) + '%' if str(int(s)) != '-1' else '-'
func_date = lambda s: s.dt.strftime('%m/%d/%Y')

## CSS Styles

In [12]:
#Zellformatierung CSS
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal; font-family: sans-serif;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #FFFFFF; color: #000000; font-family: sans-serif;'
}

td = {'selector' : 'td', 'props': 'text-align:right; font-family: sans-serif'}

## Testen der class

In [ ]:
rt = RtDuck()
rt

In [14]:
#Schließen der Verbindung
#rt.verbindung_schliessen()

In [ ]:
rt.create_table_fahrten(server = 'prod')

In [ ]:
rt.create_table_zusatz(server = 'prod')
rt.create_table_verlauf(server = 'prod')
rt.create_table_matrix(server = 'prod')

In [ ]:
rt.cursor.sql("select lineid, lineid_short, * from fahrten where datum::date = '2025-01-12' and lineid like 'de:VBN:730%'")

In [20]:
df_line_clientid = rt.cursor.sql("""select distinct lineid_short, clientid, max(datum) as max_datum
                                 from fahrten 
                                 where datum >= (current_date - interval 7 day) 
                                 group by all
                                 order by  clientid, lineid_short""").df()

In [ ]:
rt.cursor.sql( """select f.lineid_short, datum::date as datum, buendel, 
                round((count (*) filter (realtimeHasEverBeenReported = true) / count(*) ) * 100, 1) as anteil_ez  
              from fahrten f 
                join linien l on f.lineid_short = l.dlid 
              
              where datum::date = '2025-01-12'
              group by all""")

## Erstellen der Statistiken log_12_pivot und log_3_pivot

In [22]:
def sql_ebenen(ebenen):
            sql = f""" 
              select datum::date::text as datum, buendel, 
                round((count (*) filter (realtimeHasEverBeenReported = true) / count(*) ) * 100, 1) as anteil_ez  
              from fahrten f 
                join linien l on f.lineid_short = l.dlid 
              where 
                f.datum > (current_date - interval 60 day)
              and ebene in {ebenen}
              group by all"""
            
            return sql


### HTML Stil

In [23]:
#Zellformatierung CSS
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal; font-family: sans-serif;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #FFFFFF; color: #000000; font-family: sans-serif;'
}

td = {'selector' : 'td', 'props': 'text-align:right; font-family: sans-serif'}

In [24]:
ebenen = ('1+', '1', '2', 'Stadt', 'Nacht')
df = rt.cursor.sql(sql_ebenen(ebenen=ebenen)).df()
df_pivot = df.pivot(index='datum', columns='buendel', values='anteil_ez')
df_pivot.sort_values('datum', ascending=False).style.background_gradient(cmap="RdYlGn", axis = None,  vmin=0.0, vmax=95)\
    .highlight_null(color='white').format(formatter = '{:.1f}%', precision=1, na_rep='-', thousands=" ", decimal= ',').set_table_styles([index_names, headers, td])\
        .to_html('/var/www/rt_archiv/log_12_pivot.html', encoding='LATIN1')

In [25]:
ebenen = "('3')"
df = rt.cursor.sql(sql_ebenen(ebenen=ebenen)).df()
df_pivot = df.pivot(index='datum', columns='buendel', values='anteil_ez')
df_pivot.sort_values('datum', ascending=False).style.background_gradient(cmap="RdYlGn", axis = None,  vmin=0.0, vmax=95)\
    .highlight_null(color='white').format(formatter = '{:.1f}%', precision=1, na_rep='-', thousands=" ", decimal= ',').set_table_styles([index_names, headers, td])\
        .to_html('/var/www/rt_archiv/log_3_pivot.html', encoding='LATIN1')

In [ ]:
rt.anzahl_fahrten_betreiber()

In [27]:
rt.anzahl_fahrten_betreiber().to_html('/var/www/rt_archiv/anzahl_fahrten_betreiber.html', encoding='LATIN1')

## Ermitteln der Fahrten, die nur 0 Min senden bzw. im Verlauf nur 0 gespeichert wurde

In [ ]:
rt.cursor.sql(""" select * from
                (select ex_lineid, fnr,min(operday) as start, max(operday) as ende ,count(*) as count from (
                    select * from 
                        (select operday, ex_lineid, fnr, avg(dep_del) as avg_del
                        from verlauf 
                        where dep_del is not null
                        and has_rt = true
                        group by all)
                    where avg_del = 0 and ex_lineid like 'de:VBN:6__:%' and operday > (current_date - interval 28 day)
                    order by ex_lineid)               
                
                group by all
                order by count desc)
              where count > 3
              """).df()

In [29]:
rt.cursor.sql("select * from verlauf where fnr = '1630018'").df().to_excel('out/verlauf_1630018.xlsx', index=False)

In [ ]:
suffix = 'mitte'
#auswahl_linien = '680|660|N68'
auswahl_linien = '630|670|N63|N67'

df_auswahl_ohne_rt = rt.cursor.sql(f"""select * from
              (select lineshort, min(datum)::date as min_datum, max(datum)::date as max_datum, fnr, 
              count(* ) as anzahl, 
              count(* ) filter (hasRealtime  = false ) as anzahl_ohne_rt, 
              anzahl_ohne_rt / count(* ) as proz_ohne_rt
              from fahrten  
              where lineid  SIMILAR TO '.*({auswahl_linien}).*' 
              -- and hasRealtime  = false 
              and datum  >= (current_date() - interval 28 days)
              group by all
              )
              where anzahl_ohne_rt > 1
              order by proz_ohne_rt desc

              -- limit 10""").df()

df_zusatz = rt.cursor.sql(f"""select * from zusatz where lineid  SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' """).df()

ohne_rt_xl = f"out/rt_ohne_realtime_{suffix}.xlsx"
sn01 = 'ohne_rt'
with pd.ExcelWriter(ohne_rt_xl, engine='openpyxl') as writer:
    df_auswahl_ohne_rt.to_excel(writer, sheet_name=sn01, index=False)
    worksheet = writer.sheets[sn01]
    worksheet.freeze_panes = 'a2'

    worksheet.column_dimensions['B'].width = 15
    worksheet.column_dimensions['C'].width = 15
    worksheet.auto_filter.ref = worksheet.dimensions

    # Format the 'Zeit' column as date
    for cell in worksheet['B']:  # Assuming 'Zeit' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = 'YYYY-MM-DD'

    # Format the 'Zeit' column as date
    for cell in worksheet['C']:  # Assuming 'Zeit' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = 'YYYY-MM-DD'

    # Format the 'Prozent' column as percentage
    for cell in worksheet['G']:  # Assuming 'Prozent' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = '0.0%'
 
df_zusatz

In [ ]:
df_linien_quote_rt = rt.cursor.sql("""
            select * from
              ( select lineshort, min(datum)::date as min_datum, max(datum)::date as max_datum,  
                    count(* ) as anzahl, 
                    count(* ) filter (hasRealtime  = false ) as anzahl_ohne_rt, 
                    anzahl_ohne_rt / count(* ) as proz_ohne_rt
                from fahrten  
                where 
                -- and hasRealtime  = false 
                    datum  >= (current_date() - interval 28 days)
                group by all
              )
              where anzahl_ohne_rt > 1
              order by proz_ohne_rt desc

              -- limit 10""").df()

df_linien_quote_rt

In [ ]:
df_zusatz

### Auswertung Matrix nach Verlauf Zeitpunkt der Meldung

In [33]:
df_matrix = rt.cursor.sql("""select m.operatingDay::date, m.lineShortName, m.journeyId, v.index, 
                          m.stationName, m.scheduleDeparture,m.delay_minutes_arrival, m.delay_minutes_departure, m.timestamp, v.arr_del, v.dep_del
                from matrix m
                left join verlauf v on 
                          m.operatingDay = v.operday and 
                          m.lineShortName = v.lineshortname and 
                          m.journeyId = v.fnr and 
                          m.stationName = v.station_name
                where stop_cancelled = false
              and m.lineShortName = 'RS3'
              order by  m.operatingDay, m.externalLineId, m.journeyId, v.index,  m.timestamp 
              
              -- limit 20""").df()

In [ ]:
auswahl_linien = '630|670|N68|N63|N67'
df_zusatz = rt.cursor.sql(f"""
                select datum::date as datum, lineshort,lineid ,fnr,  vu 
                from zusatz 
                where                       

                    lineid SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' and 
                    -- and vu like 'Reisedienst von Rahden%' 
                    datum::date >= (current_date - interval 30 day)
                group by all 
                order by lineshort, fnr """).df()

df_zusatz

#rt.cursor.sql(f"""select * from zusatz where lineid  SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' and datum::date >= (current_date - interval 30 day)""").df()

In [ ]:
rt.cursor.sql("select min(datum )::date as min_date, max(datum)::date as amx_date, count(*) as anzahl from fahrten")

In [36]:
rt.create_vw_buendel('TN 5 CUX')

In [ ]:
rt.cursor.sql("select * from vw_buendel").df()

### Häufung von Fahrten ohne Echtzeit

In [38]:
df_fahrten_ohne_ez = rt.cursor.sql("""
              
                select datum::date as datum, ebene, lineshort , fnr, hasrealtime
               
                from vw_buendel 
                where datum >= (current_date - interval 30 day) and hasrealtime = false
                group by all
                order by ebene, lineshort, fnr
    
              """).df()

df_fahrten_ohne_ez_zusatz = df_fahrten_ohne_ez.merge(df_zusatz, left_on = ['datum', 'fnr'], right_on = ['datum', 'fnr'], how='left')
df_fahrten_ohne_ez_zusatz.query("~vu.isnull()") 

df_fahrten_ohne_ez_zusatz[['lineshort_x','datum','fnr']].groupby(['lineshort_x','fnr'], as_index=False)\
    .agg(datum_min=('datum', 'min'), datum_max=('datum', 'max'), count=('datum', 'count')).sort_values('count', ascending=False)\
    .to_excel('out/rt_fahrten_ohne_ez_zusatz.xlsx', index=False)

In [ ]:
df_fahrten_ohne_ez_zusatz.query("~vu.isnull()")

In [ ]:
interval_auswertung = 21
df_fahrten_mit_nicht_vollstaendiger_echtzeit = rt.cursor.sql(f"""
            select * from 
                (select ebene, lineshort , fnr, count(*) as anz, count(*) filter (hasRealtime) as anz_rt, 
                    (anz - anz_rt) as f_ohne_rt ,round(anz_rt/anz,2) as quote,
                    max(datum::date) filter (hasRealtime) as letzte_lieferung_echtzeit
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day)
                group by all
                order by ebene, lineshort, fnr)
            where f_ohne_rt > 1 and ebene in ('1+','1', '2') 
            order by f_ohne_rt desc                                                             
            """).df()

df_fahrten_mit_nicht_vollstaendiger_echtzeit

In [41]:
xl = 'out/nicht_vollstaendig.xlsx'
sn01 = '01 fahrten_rt_kl_100_roz'
sn02 = '02 zusatzfahrten'
sn03 = '03 ohne ez merge zusatz'

with pd.ExcelWriter(xl, engine='openpyxl') as writer: 
    df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_excel(writer, index=False, sheet_name=sn01)
    writer.book[sn01].freeze_panes = 'A2'
    writer.book[sn01].auto_filter.ref='A:H'

    df_zusatz.to_excel(writer, index=False, sheet_name=sn02)
    writer.book[sn02].freeze_panes = 'A2'
    writer.book[sn02].auto_filter.ref='A:H'

    df_fahrten_ohne_ez_zusatz.to_excel(writer, index=False, sheet_name=sn03)
    writer.book[sn03].freeze_panes = 'A2'
    writer.book[sn03].auto_filter.ref='A:H'


In [ ]:
q = rt.cursor.sql("""
                   (select 
                    datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                    count(*) filter (hasRealtime) anz_rt, round(anz_rt/ anz,2) anteil_rt, 
                    max(datum) filter (hasRealtime) letzte_lieferung
                    from vw_buendel 
                    where datum >= date_trunc('month', (date_trunc('month',current_date) - interval 1 day)::date)
                    and datum <= (date_trunc('month',current_date) - interval 1 day)::date
                  
                    group by all

                    order by datum::date)
                  """)
#q.filter("lineshort in ('S35', '350')") #mit filter einfache Abfragen

q

In [ ]:
#Abfrage für den letzten Monat
q_pivot_lm = rt.cursor.sql("""
                    pivot (select 
                            datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                            count(*) filter (hasRealtime) anz_rt, round(anz_rt/ anz,2) anteil_rt
                        from vw_buendel 
                        where datum >= date_trunc('month', (date_trunc('month',current_date) - interval 1 day)::date)
                            and datum <= (date_trunc('month',current_date) - interval 1 day)::date
                        group by all
                        )
                    on datum
                    using sum(anteil_rt)
                    group by lineshort, ebene
                    order by ebene, lineshort""")

q_pivot_lm.df().fillna('-')

## Ausgabe je Bündel als html / xlsx

### Erstellen der sortierten Bündelliste

In [44]:
list_buendel = sorted(rt.cursor.sql("select distinct buendel from linien where buendel not in ('nahsh')").df()['buendel'].to_list())

In [45]:
#Zellformatierung CSS
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal; font-family: sans-serif; font-size: 15px;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #FFFFFF; color: #000000; font-family: sans-serif; font-size: 15px;text-orientation: upright;'
}

td = {'selector' : 'td', 'props': 'text-align:right; font-family: sans-serif; font-size: 14px;'}

In [ ]:
rt.cursor.sql("""describe fahrten""")

In [ ]:
for b in list_buendel[8:9]:
    print(b, b.replace(' ', '_').lower(), replace_german_special_characters(b).replace(' ', '_').lower())

In [ ]:
func_proz = lambda s: str(int((1-s) * 1000)/10) + '%' if str(int(s)) != '-1' else '-'
func_date = lambda s: s.dt.strftime('%m/%d/%Y')

interval_auswertung = 21

date_style = NamedStyle(name="date_style", number_format="YYYY-MM-DD")
eine_nachkomma = NamedStyle(name = 'eine_nachkomma', number_format= '#,##0.0')
zwei_nachkomma = NamedStyle(name = 'eine_nachkomma', number_format= '#,##0.00')

for b in list_buendel:
    print(b, b.replace(' ', '_').lower(), replace_german_special_characters(b).replace(' ', '_').lower())

    rt.create_vw_buendel(b)
    rt.create_vw_buendel_verlauf(buendel=b)
    #Abfrage für die letzten 30 Tage
    q_pivot_lm = rt.cursor.sql(f"""
                        pivot (select 
                                datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                                count(*) filter (realtimeHasEverBeenReported ) anz_rt, round(anz_rt/ anz,2) anteil_rt
                            from vw_buendel 
                            where datum >= (current_date - interval {interval_auswertung} day)
                            group by all
                            )
                        on datum
                        using sum(anteil_rt)
                        group by lineshort, ebene
                        order by ebene, lineshort""")
    
    #Liste der Fahrten ohne Echtzeit die häufiger als 1 mal vorkommen
    df_fahrten_mit_nicht_vollstaendiger_echtzeit = rt.cursor.sql(f"""
                select * from 
                    (select ebene, lineshort , fnr, count(*) as anz, count(*) filter (hasRealtime) as anz_ez, 
                    (anz - anz_ez) as fahrten_ohne_ez ,round(anz_ez/anz,2) as quote,
                    max(datum::date) filter (realtimeHasEverBeenReported ) as letzte_lieferung_echtzeit
                    from vw_buendel 
                    where datum >= (current_date - interval {interval_auswertung} day)
                    group by all
                    order by ebene, lineshort, fnr)
                where fahrten_ohne_ez > 1 and ebene in ('1+','1', '2','Nacht') 
                    order by fahrten_ohne_ez desc                                                             
                
                """).df()
    
    df_fahrten_ohne_ez = rt.cursor.sql(f"""              
                select datum::date as datum, ebene, lineshort , fnr, hasrealtime               
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day) and realtimeHasEverBeenReported  = false
                group by all
                order by ebene, lineshort, fnr
    
              """).df()
    
    df_fahrten_gesamt = rt.cursor.sql(f"""              
                select *               
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day) 
                order by ebene, lineshort, fnr
    
              """).df()
    
    html_zusatz_table = 'html/pre_zusatz.html'
    df_fahrten_ohne_ez_zusatz = df_fahrten_ohne_ez.merge(df_zusatz, left_on = ['datum', 'fnr'], right_on = ['datum', 'fnr'], how='left')
    df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").to_html(html_zusatz_table, index=False)

    html_pre_table = 'html/pre_table.html'
    df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_html(html_pre_table, index=False)

    html_pre_pivot = 'html/pre_pivot.html'
    q_pivot_lm.df().style.background_gradient(cmap="RdYlGn", axis = None,  vmin=0.5, vmax=1).highlight_null(color='white')\
        .format( precision=2, na_rep='-', thousands=" ")\
        .highlight_null(color='white')\
        .set_table_styles([index_names, headers, td])\
        .to_html(html_pre_pivot)
    
    # Save the HTML table to a file (optional)   
    with open(html_pre_pivot, 'r') as file:
        html_pre_pivot = file.read()
    
    # Load the HTML page template
    with open('html/template.html', 'r') as file:
        html_template = file.read()

    # Insert the HTML table into the template
    title = f"Echtzeitquote Bündel {b} je Linie erstellt: {dt.datetime.now().strftime('%d.%m.%Y %H:%M')}" 
    html_page = html_template.replace('{{ html_pivot }}', html_pre_pivot).replace('{{ html_title }}', title)

    if df_fahrten_mit_nicht_vollstaendiger_echtzeit.shape[0] > 0:
        html_page = html_page.replace('{{ html_table }}', df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_html(index=False))
    else:
        html_page = html_page.replace('{{ html_table }}', "Keine Häufung Fahrten ohne Echtzeit")

    if df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").shape[0] > 0:
        html_page = html_page.replace('{{ html_table_zusatz }}', df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").to_html(index=False))
    else:
        html_page = html_page.replace('{{ html_table_zusatz }}', "Keine Zusatzfahrten mit gleicher Fahrtnummer")

    # Save the combined HTML page to a file
    html_combined = f"/var/www/rt_archiv/buendel/rt_{replace_german_special_characters(b).replace(' ', '_').lower()}.html"


    with open(html_combined, 'w') as file:
        file.write(html_page)


    #Ausgabe der wichtigen Ergebnisse als Excel
    xl = f"buendel_stat/{replace_german_special_characters(b).replace(' ', '_').lower()}_stat.xlsx"

    sn00 = '01 hilfe'
    sn01 = '02 statistik ebene'
    sn02 = '03 statistik pivot'
    sn03 = '04 fahrten gesamt'
    sn04 = '05 verlauf' #nicht im Stadtverkehr

    with pd.ExcelWriter(xl, engine='openpyxl') as writer:
        writer.book.add_named_style(date_style)

        df_vorfaelle = rt.df_vorfaelle_echtzeit()
        df_vorfaelle.to_excel(writer, index=False, sheet_name=sn01)
        writer.book[sn01].freeze_panes = 'A2'
        writer.book[sn01].auto_filter.ref='A:F'
        for row in writer.book[sn01].iter_rows(min_row=2, min_col=1, max_col=1):
                for cell in row:
                    cell.style = date_style

        for row in writer.book[sn01].iter_rows(min_row=2, min_col=5, max_col=5):
                for cell in row:
                    cell.style = zwei_nachkomma
        writer.book[sn01][f"F{df_vorfaelle.shape[0]+3}"] = f"=SUBTOTAL(9, F2:F{df_vorfaelle.shape[0]+1})"
        writer.book[sn01].column_dimensions['A'].width = 15

        q_pivot_lm.df().to_excel(writer, index=True, sheet_name=sn02)
        writer.book[sn02].freeze_panes = 'A2'
        writer.book[sn02].auto_filter.ref='A:H'

        df_fahrten_gesamt.to_excel(writer, index=False, sheet_name=sn03)
        writer.book[sn03].freeze_panes = 'A2'
        writer.book[sn03].auto_filter.ref='A:N'
        writer.book[sn03].column_dimensions['A'].width = 15
        for row in writer.book[sn03].iter_rows(min_row=2, min_col=1, max_col=1):
                for cell in row:
                    cell.style = date_style

        #Verlauf nicht im Verkehr
        if b not in ('HB Bus', 'HB Tram', 'BHV', 'DEL', 'OL Stadt'):

            rt.cursor.sql("""from vw_buendel_verlauf""").df().to_excel(writer, index=False, sheet_name=sn04)
            writer.book[sn04].freeze_panes = 'A2'
            writer.book[sn04].auto_filter.ref='A:J'
            writer.book[sn04].column_dimensions['A'].width = 15
            writer.book[sn04].column_dimensions['F'].width = 30
            for row in writer.book[sn04].iter_rows(min_row=2, min_col=1, max_col=1):
                for cell in row:
                    cell.style = date_style

    # Öffnen des Workbooks und Anwenden der Formatierung
    wb = openpyxl.load_workbook(xl)

    #Erstellen des Hilfeblattes an erster Position
    wb.create_sheet(sn00, index=0)
    sheet = wb[sn00]
    sheet['A1'] = f"Erstellt: {dt.datetime.now().strftime('%Y-%m-%d %H:%M')}"
    sheet['A2'] =  "Erläuterung der Werte in der Tabelle"
    sheet['A3'] = f"Blatt {sn01} enthält die Echtzeitquote der Ebenen des Bündels {b} für die letzten {interval_auswertung} Tage"
    sheet['A4'] = f"Blatt {sn02} enthält die Echtzeitquote der Linien des Bündels {b} für die letzten {interval_auswertung} Tage"
    sheet['A5'] = f"Blatt {sn03} filterbare Liste der Fahrten {b} für die letzten {interval_auswertung} Tage"

    wb.save(xl)       

### Upload nach Redmine

In [ ]:
list_excel = glob.glob('buendel_stat/*.xlsx')
list_redmine = pd.read_csv('input/folder_vms.csv', sep=';', quotechar="'")['buen'].to_list()
df_redmine  = pd.read_csv('input/folder_vms.csv', sep=';', quotechar="'")

for b in list_buendel:
    b_clean = replace_german_special_characters(b).replace(' ', '_').lower()
    folder_file_name = os.path.join('buendel_stat', f"{b_clean}_stat.xlsx") 
    file_name =  f"{b_clean}_stat.xlsx"

    shutil.copyfile(folder_file_name, f'/var/www/rt_archiv/{file_name}')

    if b_clean not in list_redmine:
        print(f"not in {b_clean}")
    
    else:
        print(f"{b} in {b_clean}")
        project_url = df_redmine.query(f"buen == '{b_clean}'").reset_index().at[0, 'project_url']
        folder_id = df_redmine.query(f"buen == '{b_clean}'").reset_index().at[0, 'folder_id']

        delete_upload_dmsf(project_url=project_url, folder_id=folder_id, file_name=file_name, folder_file_name=folder_file_name)
        logging.info(f"{folder_file_name} hochgeladen")


In [34]:
logging.info(f"Anzahl Fahrten gesamt {rt.anzahl_fahrten()}")

In [ ]:
rt.verbindung_schliessen()